## C2. Spatial Cluster Analysis

**Description**  
This section conducts comprehensive spatial analysis to identify geographic patterns and clustering in hospital-level AI adoption across the United States. The analysis employs multiple complementary approaches to characterize spatial autocorrelation, identify statistically significant clusters, and examine regional disparities in AI implementation.

**Purpose**  
To explore patterns in hospital-level AI adoption across geographic regions (state or census division), aiding interpretation of implementation disparities.


### 1 load necessary libraries, functions and preprocessed data 

In [ ]:

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
import scipy.spatial as spatial
from scipy.stats import zscore
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import os
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Import functions
import sys
sys.path.append('../')
from calculate_scores import create_union_aipred_row, apply_ai_scores_to_dataframe

# Load data
AHA_master = pd.read_csv('../../data/AHA_master_external_data.csv', low_memory=False)

# Create aipred_it_union separately (your choice, works perfectly)
AHA_master['aipred_it_union'] = AHA_master.apply(create_union_aipred_row, axis=1)

# Use the apply function for all other scores
AHA_master2 = apply_ai_scores_to_dataframe(AHA_master)
AHA_IT = AHA_master2[AHA_master2['id_it'].notna()]


In [ ]:
# load STATE shape file 
os.environ['SHAPE_RESTORE_SHX'] = 'YES'
states = gpd.read_file('../../temp_shp/cb_2018_us_state_500k.shp')

### 2 Data engineering 

In [ ]:
# Remove rows with missing or invalid coordinates
AHA_IT2 = AHA_IT.dropna(subset=['lat_as', 'long_as'])

# Filter out invalid coordinates
valid_coords = (
    (AHA_IT2['lat_as'] != 0) & 
    (AHA_IT2['long_as'] != 0) &
    (AHA_IT2['lat_as'] >= -90) & 
    (AHA_IT2['lat_as'] <= 90) &
    (AHA_IT2['long_as'] >= -180) & 
    (AHA_IT2['long_as'] <= 180)
)
AHA_IT2 = AHA_IT2[valid_coords]

# Create GeoDataFrame
hospitals = gpd.GeoDataFrame(
    AHA_IT2, 
    geometry=gpd.points_from_xy(AHA_IT2.long_as, AHA_IT2.lat_as),
    crs="EPSG:4326"
)


In [ ]:
# Filter hospitals with valid coordinates and implementation scores
valid_hospitals = hospitals.dropna(subset=['long_as', 'lat_as', 'aipred_it'])
valid_geo_hospitals = hospitals.dropna(subset=['long_as', 'lat_as'])
# Create a GeoDataFrame
hospitals_gdf = gpd.GeoDataFrame(
    valid_hospitals, 
    geometry=gpd.points_from_xy(valid_hospitals.long_as, valid_hospitals.lat_as),
    crs="EPSG:4326" #geographic coordinate system using latitude and longitude
)

# Create a GeoDataFrame
geo_hospitals_gdf = gpd.GeoDataFrame(
    valid_geo_hospitals, 
    geometry=gpd.points_from_xy(valid_geo_hospitals.long_as, valid_geo_hospitals.lat_as),
    crs="EPSG:4326" #geographic coordinate system using latitude and longitude
)


In [ ]:
# Convert to a projected CRS for accurate distance calculations
hospitals_gdf_projected = hospitals_gdf.to_crs(epsg=3857) # projected coordinate system using flat, 2D plane to represent Earth's surface 
geo_hospitals_gdf_projected = geo_hospitals_gdf.to_crs(epsg=3857) # projected coordinate system using flat, 2D plane to represent Earth's surface 


geo_hospitals_gdf_projected['ML_implementation_score'] = geo_hospitals_gdf_projected['aipred_it_union'].map({
    1: 3,  # ML gets highest score
    2: 2,  # Non-ML gets middle score
    3: 1,  # Neither gets lowest score
    4: 1,   # Don't know gets lowest score
    None: 0,
    0: 0 
})
geo_hospitals_gdf_projected.shape


### 3 AI/ML barplot (across census division)

In [ ]:
# Create a mapping of states to regions
# Create a mapping of states to census divisions based on provided image
state_to_division = {
    # Division 1: New England
    'ME': 'New England', 'NH': 'New England', 'VT': 'New England', 
    'MA': 'New England', 'RI': 'New England', 'CT': 'New England',
    
    # Division 2: Mid Atlantic
    'NY': 'Mid Atlantic', 'NJ': 'Mid Atlantic', 'PA': 'Mid Atlantic',
    
    # Division 3: South Atlantic
    'DE': 'South Atlantic', 'MD': 'South Atlantic', 'DC': 'South Atlantic',
    'VA': 'South Atlantic', 'WV': 'South Atlantic', 'NC': 'South Atlantic',
    'SC': 'South Atlantic', 'GA': 'South Atlantic', 'FL': 'South Atlantic',
    
    # Division 4: East North Central
    'OH': 'East North Central', 'IN': 'East North Central', 'IL': 'East North Central',
    'MI': 'East North Central', 'WI': 'East North Central',
    
    # Division 5: East South Central
    'KY': 'East South Central', 'TN': 'East South Central', 
    'AL': 'East South Central', 'MS': 'East South Central',
    
    # Division 6: West North Central
    'MN': 'West North Central', 'IA': 'West North Central', 'MO': 'West North Central',
    'ND': 'West North Central', 'SD': 'West North Central', 'NE': 'West North Central',
    'KS': 'West North Central',
    
    # Division 7: West South Central
    'AR': 'West South Central', 'LA': 'West South Central', 
    'OK': 'West South Central', 'TX': 'West South Central',
    
    # Division 8: Mountain
    'MT': 'Mountain', 'ID': 'Mountain', 'WY': 'Mountain', 'CO': 'Mountain',
    'NM': 'Mountain', 'AZ': 'Mountain', 'UT': 'Mountain', 'NV': 'Mountain',
    
    # Division 9: Pacific
    'WA': 'Pacific', 'OR': 'Pacific', 'CA': 'Pacific', 
    'AK': 'Pacific', 'HI': 'Pacific',
    
    # Territories
    'PR': 'Territories', 'GU': 'Territories', 'VI': 'Territories', 
    'AS': 'Territories', 'MP': 'Territories'
}

# Add census division column to the dataframe
hospitals_gdf_projected['division'] = hospitals_gdf_projected['mstate_it'].map(state_to_division)
geo_hospitals_gdf_projected['division'] = geo_hospitals_gdf_projected['mstate_it'].map(state_to_division)

# Loop through each census division 
divisions = [
    'New England', 'Mid Atlantic', 'South Atlantic', 
    'East North Central', 'East South Central', 'West North Central',
    'West South Central', 'Mountain', 'Pacific'
]


In [ ]:
# Use  existing ai_types
ai_types = [3, 2, 1]  

# Map the numeric AI types to labels for better readability
ai_type_labels = {
    3: 'ML',
    2: 'Non-ML Predictive Model',
    1: 'Do not know/Neither',
    0: 'No Response'  # Assuming 0 or NaN is used for no response
}

# Create a DataFrame to hold the distribution of AI types by division
division_ai_counts = pd.DataFrame(index=divisions, columns=ai_types + [0])  # Include 0 for no response
# Fill the DataFrame with counts
for division in divisions:
    geo_division_data = geo_hospitals_gdf_projected[geo_hospitals_gdf_projected['division'] == division]
    
    # Count no responses (either 0 or NaN)
    no_response_count = len(geo_division_data[geo_division_data['ML_implementation_score'].isna() | 
                                         (geo_division_data['ML_implementation_score'] == 0)])
    division_ai_counts.loc[division, 0] = no_response_count
    
    # Count other AI types
    for ai_type in ai_types:
        division_ai_counts.loc[division, ai_type] = len(geo_division_data[geo_division_data['ML_implementation_score'] == ai_type])
division_ai_counts

# Calculate total hospitals per division
division_totals = division_ai_counts.sum(axis=1)

# Calculate percentages
division_ai_dist = division_ai_counts.div(division_ai_counts.sum(axis=1), axis=0) * 100

# Calculate grid dimensions
n_divisions = len(divisions)
n_cols = 3  # Number of columns in the grid
n_rows = 3

In [ ]:


# Map the numeric AI types to labels for better readability
ai_type_labels = {
    3: 'AI/ML',
    2: 'Non-AI/ML Predictive Model',
    1: 'Do not know/Neither',
    0: 'No Response'
}

# Rename columns for better readability
division_ai_counts = division_ai_counts.rename(columns=ai_type_labels)

# Calculate percentages
percentages = division_ai_counts.div(division_ai_counts.sum(axis=1), axis=0) * 100

# Sort divisions by total
total_hospitals = division_ai_counts.sum(axis=1)
sorted_divisions = division_ai_counts.loc[total_hospitals.sort_values(ascending=False).index]
sorted_percentages = percentages.loc[total_hospitals.sort_values(ascending=False).index]

# Define publication-quality colors
colors = ['#3366CC', '#66CCEE', '#EEEEFF', '#CCCCCC']

# Create figure with a higher DPI for better quality
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)

# Create stacked bar chart
bars = sorted_divisions.plot(kind='bar', stacked=True, color=colors, ax=ax, width=0.7, edgecolor='white', linewidth=0.5)

# Add percentage labels on top of each segment
for i, division in enumerate(sorted_divisions.index):
    cumulative_height = 0
    for j, col in enumerate(sorted_divisions.columns):
        height = sorted_divisions.loc[division, col]
        pct = sorted_percentages.loc[division, col]
        
        if height > 0:  # Only label non-zero values
            # Position at the top of each segment
            y_pos = cumulative_height + height
            
            # For small percentages, only show if they're at least 1%
            if pct >= 1:
                # Position text at the top of each segment
                ax.text(i, cumulative_height + (height * 0.5), f'{pct:.1f}%', 
                        ha='center', va='center', fontsize=9,
                        color='black', fontweight='bold')
            
        cumulative_height += height

ax.set_xlabel('Census Division', fontsize=14, labelpad=10, color='black')
ax.set_ylabel('Number of Hospitals', fontsize=14, labelpad=10, color='black')
ax.tick_params(axis='both', which='major', labelsize=12, colors='black')

plt.xticks(rotation=45, ha='right', color='black')
plt.yticks(color='black')

# Calculate combined percentage of ML and Non-ML Predictive Model
combined_percentage = sorted_percentages['AI/ML'] + sorted_percentages['Non-AI/ML Predictive Model']

# Add combined percentage labels above the total count
for i, division in enumerate(sorted_divisions.index):
    total = total_hospitals[division]
    combined_pct = combined_percentage[division]
    # Position the combined percentage above the total count
    plt.text(i, total + 20, f'Model: {combined_pct:.1f}%', 
             ha='center', fontsize=8,  # Smaller font size
             fontweight='normal',      # Normal weight instead of bold
             color='#666666')          # Gray color


plt.tight_lat()
plt.show()


### 4 Hospital clustering - nearest neighbor 

In [ ]:

# nearest neighbor analysis (without relying on pointpats)
print("Performing Custom Nearest Neighbor Analysis...")

# Extract coordinates
coords = np.vstack((geo_hospitals_gdf_projected.geometry.x, geo_hospitals_gdf_projected.geometry.y)).T

# Calculate distances between all pairs of points
kdtree = spatial.KDTree(coords)
distances, indices = kdtree.query(coords, k=2)  # k=2 to get the nearest neighbor (first one is self)
mean_observed_nn_distance = np.mean(distances[:, 1])

# Calculate the area - use the bounding box as an approximation
x_min, y_min, x_max, y_max = geo_hospitals_gdf_projected.total_bounds
area = (x_max - x_min) * (y_max - y_min)

# Calculate point density
n = len(coords)
density = n / area

# Expected mean distance for random distribution
mean_expected_nn_distance = 0.5 / np.sqrt(density)

# Calculate nearest neighbor ratio
nn_ratio = mean_observed_nn_distance / mean_expected_nn_distance

# Standard error
se = 0.26136 / np.sqrt(n * density)

# Z-score
z_score = (mean_observed_nn_distance - mean_expected_nn_distance) / se

# Calculate approximate p-value
from scipy.stats import norm
p_value = 2 * (1 - norm.cdf(abs(z_score)))  # two-tailed test

print(f"Nearest Neighbor Ratio: {nn_ratio:.3f}")
print(f"z-score: {z_score:.3f}")
print(f"p-value: {p_value:.3f}")

if z_score < -1.96:
    print("Hospitals show significant clustering (p < 0.05)")
elif z_score > 1.96:
    print("Hospitals show significant dispersion (p < 0.05)")
else:
    print("Hospitals show random spatial pattern")




### 5 Moran's I spatial autocorrelation - AI and model implementation measures 

#### 5.1 moran's I function

In [ ]:
def calculate_morans_i(values, coords, k=6):
    """Calculate Moran's I"""
    import numpy as np
    from libpysal.weights import KNN
    from esda.moran import Moran
    
    # Convert inputs to numpy arrays
    values = np.asarray(values).flatten()
    coords = np.asarray(coords)
    
    # Handle missing values
    valid_idx = ~np.isnan(values)
    if not np.all(valid_idx):
        values = values[valid_idx]
        coords = coords[valid_idx]
    
    n = len(values)
    if n <= 1 or np.var(values) == 0:
        return {'moran_i': np.nan, 'expected_i': np.nan, 'z_score': np.nan, 'p_value': np.nan, 'n': n}
    
    try:
        # Create k-nearest neighbor weights using PySAL
        w = KNN.from_array(coords, k=k)
        
        # Calculate Moran's I using PySAL
        moran = Moran(values, w)
        
        return {
            'moran_i': moran.I,
            'expected_i': moran.EI,
            'z_score': moran.z_norm,
            'p_value': moran.p_norm,
            'n': n
        }
    
    except Exception as e:
        print(f"PySAL error: {e}")
        return {'moran_i': np.nan, 'expected_i': np.nan, 'z_score': np.nan, 'p_value': np.nan, 'n': n}

#### 5.2 US moran's I 

In [ ]:
# Global Moran's I Spatial Autocorrelation Analysis

# Extract coordinates and target variable
coords = np.vstack((hospitals_gdf_projected.geometry.x, hospitals_gdf_projected.geometry.y)).T
values = hospitals_gdf_projected['ai_base_score_imputed'].values



In [ ]:
# Extract coordinates 
coords = np.vstack((geo_hospitals_gdf_projected.geometry.x, geo_hospitals_gdf_projected.geometry.y)).T

# Define the variables to analyze 
ai_variables = [
    'ai_base_score_imputed', 
    'ai_base_breadth_score_imputed', 
    'ai_base_dev_score_imputed', 
    'ai_base_eval_score_2023_imputed',
    'ai_base_eval_score_2024_imputed', 
    'llm_readiness_score'
]

# Create a list to store results 
results_list = []

# Calculate Moran's I for each variable 
for var in variables:
    try:
        # Get values for this variable
        values = geo_hospitals_gdf_projected[var].values
        
        # Calculate Moran's I
        result = calculate_morans_i(values, coords, k=6)
        
        # Determine pattern 
        if np.isnan(result['p_value']):
            pattern = "Invalid data"
        elif result['p_value'] < 0.05:
            if result['moran_i'] > result['expected_i']:
                pattern = "Significant clustering"
            else:
                pattern = "Significant dispersion"
        else:
            pattern = "Random distribution"
        
        # Add results to list 
        results_list.append({
            'Variable': var,
            'Moran I': result['moran_i'],
            'Expected I': result['expected_i'],
            'z-score': result['z_score'],
            'p-value': result['p_value'],
            'Pattern': pattern,
            'n': result['n']
        })
    
    except Exception as e:
        results_list.append({
            'Variable': var,
            'Moran I': np.nan,
            'Expected I': np.nan,
            'z-score': np.nan,
            'p-value': np.nan,
            'Pattern': f"Error: {str(e)}",
            'n': np.nan
        })

# Create DataFrame 
moran_results_df = pd.DataFrame(results_list)

# Format the numeric columns 
moran_results_df['Moran I'] = moran_results_df['Moran I'].round(4)
moran_results_df['Expected I'] = moran_results_df['Expected I'].round(4)
moran_results_df['z-score'] = moran_results_df['z-score'].round(4)
moran_results_df['p-value'] = moran_results_df['p-value'].round(10)

# Display the DataFrame 
print("\nMoran's I Analysis Results (k=6 neighbors):")
moran_results_df

### 6. DBSCAN - cluster identification 

#### 6.1 DBSCAN function 

In [ ]:
def adaptive_dbscan_clustering(hospitals_gdf, division, output_col, eps=0.35, min_samples=2, k=6):
    """
    Perform DBSCAN clustering with adaptive feature weights based on local density.
    
    Parameters:
    - hospitals_gdf: GeoDataFrame with hospital data
    - division: Census division to analyze
    - output_col: Column name for the ML score to use
    - eps: DBSCAN epsilon parameter
    - min_samples: DBSCAN min_samples parameter
    - k: Number of neighbors to consider for density estimation
    """
    # Filter data for this division
    division_data = hospitals_gdf[hospitals_gdf['division'] == division].copy()
    
    # Check if we have the target column
    if output_col not in division_data.columns:
        print(f"Error: Column '{output_col}' not found for division '{division}'")
        return pd.DataFrame(), []
    
    # Handle missing values - drop rows with NaN in the target column or coordinates
    division_data = division_data.dropna(subset=[output_col, 'geometry'])
    
    # Check if we have enough data after dropping NaNs
    if len(division_data) < min_samples:
        print(f"Skipping {division}: too few hospitals with valid data ({len(division_data)})")
        return pd.DataFrame(), []
    
    # Print diagnostic info
    print(f"  Processing {division} with {len(division_data)} hospitals (valid data)")
    
    # Calculate local density using k-nearest neighbors
    coords = np.column_stack([division_data.geometry.x, division_data.geometry.y])
    
    # Ensure coordinates don't have NaNs
    if np.isnan(coords).any():
        print(f"  Warning: Some coordinates contain NaN values. Removing affected points.")
        valid_idx = ~np.isnan(coords).any(axis=1)
        division_data = division_data.iloc[valid_idx].copy()
        coords = coords[valid_idx]
        
        if len(division_data) < min_samples:
            print(f"  Skipping {division}: too few hospitals after removing NaN coordinates")
            return pd.DataFrame(), []
    
    # Ensure ML scores don't have NaNs
    values = division_data[output_col].values
    if np.isnan(values).any():
        print(f"  Warning: '{output_col}' contains NaN values. Removing affected points.")
        valid_idx = ~np.isnan(values)
        division_data = division_data.iloc[valid_idx].copy()
        coords = coords[valid_idx]
        values = values[valid_idx]
        
        if len(division_data) < min_samples:
            print(f"  Skipping {division}: too few hospitals after removing NaN values")
            return pd.DataFrame(), []
    
    # Adjust k if necessary
    actual_k = min(k, len(division_data)-1)
    if actual_k < k:
        print(f"  Note: Adjusted k from {k} to {actual_k} due to small sample size")
    
    # Fit nearest neighbors
    nn = NearestNeighbors(n_neighbors=actual_k+1)  # +1 because point is its own neighbor
    nn.fit(coords)
    distances, _ = nn.kneighbors(coords)
    
    # Average distance to k nearest neighbors (excluding self)
    avg_distances = distances[:, 1:].mean(axis=1)
    
    # Convert to density: smaller distances = higher density
    density = 1 / (avg_distances + 1e-10)  # Add small constant to avoid division by zero
    
    # Higher density areas get HIGHER weights
    min_density, max_density = np.percentile(density, [5, 95])  # Using percentiles to avoid outliers
    normalized_density = (density - min_density) / (max_density - min_density + 1e-10)
    normalized_density = np.clip(normalized_density, 0, 1)  # Clip to [0, 1] first
    weights = 0.5 + 1.5 * normalized_density  # Scale to [0.5, 2.0] - higher density gets higher weights
    
    # Store the weights for later analysis
    division_data['ml_score_weight'] = weights
    
    # Create weighted features: higher weight amplifies ML scores in dense areas
    X = np.column_stack([
        coords,  # x, y coordinates
        division_data[output_col].values * weights  # Adaptive weighting - FIXED
    ])
    
    # Final check for NaNs
    if np.isnan(X).any():
        print(f"  Error: Feature matrix still contains NaN values after processing")
        print(f"  NaN counts: {np.isnan(X).sum(axis=0)}")
        return pd.DataFrame(), []
    
    # Standardize the weighted features
    X_scaled = StandardScaler().fit_transform(X)
    
    # Run DBSCAN
    try:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        cluster_labels = dbscan.fit_predict(X_scaled)
        
        # Store labels in the dataframe
        division_data['cluster'] = cluster_labels
        
        # Create cluster summary
        clusters_summary = []
        if len(set(cluster_labels)) > 1:  # If we have actual clusters
            for cluster_id in sorted(set(cluster_labels)):
                if cluster_id == -1:
                    continue  # Skip noise points
                    
                cluster_hospitals = division_data[division_data['cluster'] == cluster_id]
                
                # Calculate ML stats
                ml_mean = cluster_hospitals[output_col].mean()
                ml_std = cluster_hospitals[output_col].std() if len(cluster_hospitals) > 1 else 0
                
                clusters_summary.append({
                    'division': division,
                    'cluster_id': f"{division}-{cluster_id}",
                    'n_hospitals': len(cluster_hospitals),
                    'pct_of_division': len(cluster_hospitals) / len(division_data) * 100,
                    'ml_mean': ml_mean,
                    'ml_std': ml_std,
                    'ml_min': cluster_hospitals[output_col].min(),
                    'ml_max': cluster_hospitals[output_col].max(),
                    'avg_density_weight': cluster_hospitals['ml_score_weight'].mean(),  # Renamed for clarity
                    'density_weight_std': cluster_hospitals['ml_score_weight'].std() if len(cluster_hospitals) > 1 else 0
                })
        
        # Count results
        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        n_noise = list(cluster_labels).count(-1)
        
        print(f"  Results: {n_clusters} clusters, {n_noise} noise points ({n_noise/len(cluster_labels)*100:.1f}%)")
        print(f"  ML score weights - min: {weights.min():.2f}, max: {weights.max():.2f}, avg: {weights.mean():.2f}")
        print(f"  Weight interpretation: Higher values = denser areas get amplified ML scores")
        
        return division_data, clusters_summary
    
    except Exception as e:
        print(f"  Error during DBSCAN: {str(e)}")
        return pd.DataFrame(), []

In [ ]:
coords = np.column_stack([hospitals_gdf_projected.geometry.x, hospitals_gdf_projected.geometry.y])
ml_scores = hospitals_gdf_projected['ai_base_score_imputed'].values

In [ ]:
# Regional DBSCAN Clustering Analysis
# Applies spatial clustering to hospital data by census division

# Initialize storage for results
all_results = []
all_clusters_summary = []
modified_data = pd.DataFrame()

# Define analysis regions
regions = [
    'New England', 'Mid Atlantic', 'South Atlantic', 
    'East North Central', 'East South Central', 'West North Central',
    'West South Central', 'Mountain', 'Pacific'
]

target_metric = 'ai_base_score_imputed'



#### 6.2 DBSCAN ai base score 

In [ ]:
# DBSCAN Clustering Analysis by Census Division
# Applies adaptive clustering to each geographic region

# Initialize storage
all_clusters_summary = []
modified_data = pd.DataFrame()


target_column = 'ai_base_score_imputed'

# Apply clustering to each division
for division in divisions:
    if division not in hospitals_gdf_projected['division'].unique():
        continue
    
    # Run adaptive DBSCAN
    division_result, clusters_summary = adaptive_dbscan_clustering(
        hospitals_gdf_projected, 
        division, 
        output_col=target_column,
        eps=0.35, 
        min_samples=2,
        k=6
    )
    
    if not division_result.empty:
        modified_data = pd.concat([modified_data, division_result])
        all_clusters_summary.extend(clusters_summary)


#### 6.3 DBSCAN secondary model implmentation measures 

In [ ]:
# Initialize collection variables
all_results = []
all_clusters_summary = []
modified_hospital_data = pd.DataFrame()  # Empty DataFrame to collect results

# Define census divisions
divisions = [
    'New England', 'Mid Atlantic', 'South Atlantic', 
    'East North Central', 'East South Central', 'West North Central',
    'West South Central', 'Mountain', 'Pacific'
]

# Define the target column
target_column = 'ai_base_score_imputed'

print(f"DBSCAN Clustering Analysis for {target_column} by Census Division")
print("=" * 60)

# Apply clustering to each division
for division in divisions:
    print(f"\nAnalyzing {division} division...")
    
    # Check if this division exists in the data
    if division not in hospitals_gdf_projected['division'].unique():
        print(f"  Warning: No hospitals found in '{division}' division")
        continue
    
    # Apply clustering
    division_result, clusters_summary = adaptive_dbscan_clustering(
        hospitals_gdf_projected, 
        division, 
        output_col=target_column,
        eps=0.35, 
        min_samples=2,
        k=6
    )
    
    # Only append if we got results
    if not division_result.empty:
        modified_hospital_data = pd.concat([modified_hospital_data, division_result])
        all_clusters_summary.extend(clusters_summary)

# Convert cluster summaries to DataFrame for analysis
if all_clusters_summary:
    # Create DataFrame from list
    clusters_df = pd.DataFrame(all_clusters_summary)
    
    # Summary statistics by division
    division_summary = clusters_df.groupby('division').agg(
        Number_of_Clusters=('cluster_id', 'count'),
        Total_Hospitals=('n_hospitals', 'sum'),
        Avg_Cluster_Size=('n_hospitals', 'mean'),
        Avg_Score=('ml_mean', 'mean'),
        Min_Score=('ml_min', 'min'),
        Max_Score=('ml_max', 'max')
    ).reset_index()
    
    # Format the numeric columns
    division_summary['Avg_Cluster_Size'] = division_summary['Avg_Cluster_Size'].round(1)
    division_summary['Avg_Score'] = division_summary['Avg_Score'].round(2)
    division_summary['Min_Score'] = division_summary['Min_Score'].round(2)
    division_summary['Max_Score'] = division_summary['Max_Score'].round(2)
    
    # Overall summary as a single row DataFrame
    overall_summary = pd.DataFrame([{
        'Division': 'TOTAL',
        'Number_of_Clusters': len(clusters_df),
        'Total_Hospitals': clusters_df['n_hospitals'].sum(),
        'Avg_Cluster_Size': round(clusters_df['n_hospitals'].mean(), 1),
        'Avg_Score': round(clusters_df['ml_mean'].mean(), 2),
        'Min_Score': round(clusters_df['ml_min'].min(), 2),
        'Max_Score': round(clusters_df['ml_max'].max(), 2)
    }])
    
    # Display summary tables
    print("\nDivision Summary:")
    display(division_summary)
    
    print("\nOverall Summary:")
    display(overall_summary)
else:
    print("No clusters were identified across any division")
